## Какво са прототипите?

В JavaScript всеки обект е допълнен с неявна препратка към друг обект, наречен негов **прототип**. Когато се опитате да достъпите някое свойство (на обект), виртуалната машина се опитва да го открие в самия обект. Ако не го намери, продължава да търси в прототипа на обекта, след това в прототипа на прототипа (тъй като и прототипа е обект) и така нататък - образувайки **прототипна верига**.

Мислете за това като за наследяване в родословно дърво: ако нямате нещо, можете да го заемете от родителя си, а ако той го няма, от неговия родител и т.н.

Можем да изследваме това с ```Object.getPrototypeOf(obj)```

In [ ]:
// Всеки обект има прототип

let obj = { a: 1 };
console.log(Object.getPrototypeOf(obj));  // Object.prototype

// Масивите имат Array.prototype
let arrr = [1, 2, 3];
console.log(Object.getPrototypeOf(arrr));  // Array.prototype

// Функциите имат Function.prototype
function fn() {}
console.log(Object.getPrototypeOf(fn));   // Function.prototype

# Наследяване с прототипи

Разбиране на прототипно-базирания модел на наследяване в JavaScript.

## 1. Примери с Object.create()

Преди да се потопим в прототипите, нека изследваме `Object.create()` и боравенето с прототипи - фундаментални концепции за разбиране на модела на наследяване в JavaScript.

### Основни действия при боравене с прототипи

**Забележка:** В Deno (който използва този notebook), прякото присвояване към `__proto__` не работи както би се очаквало. Правилният начин да зададете прототипа на обект е с използване на `Object.setPrototypeOf()`, който работи еднозначно както в Node.js, така и в Deno.

In [ ]:
let obj = { cook: "Баба", meal: "Салата" };
let othObj = { value: 10 };

// Правилен начин за задаване на прототип (работи както в Node.js, така и в Deno)
Object.setPrototypeOf(othObj, obj);

console.log(othObj.value);  // 10 (собствено свойство)
console.log(othObj.cook);   // "Баба" (наследено от obj)
console.log(othObj.meal);   // "Салата" (наследено от obj)

// Забележка: В Node.js, othObj.__proto__ = obj също работи, но е остаряло
// и не работи в строгия режим на Deno

### Допълване на вградени прототипи (Използвайте с внимание!)

In [ ]:
// Добавяне на метод към Array.prototype
Array.prototype.oddCnt = function () {
    return this.filter( el => el % 2).length;
};

console.log([].oddCnt.call([1, 2, 3, 5, 6, 7]));  // 4 (четири нечетни числа)

### Object.create() и мета-описание на свойствата

In [1]:
let grandp = {
    name: "Баба",
    age: 105,
};

// Създаване на обект с grandp като прототип и дефиниране на свойства
const parent = Object.create(grandp, {
    name: {
        writable: true,
        configurable: true,
        value: "Котка",
    },
});

console.log(parent.name);  // "Котка" (собствено свойство)
console.log(parent.age);   // 105 (наследено от дядото)

Котка
105


### Object.create() с Data и свойства с контролиран достъп (getters/setters)

In [ ]:
let o = Object.create(Object.prototype, {
    
    foo: {      // foo е обикновено data свойство
        writable: true,
        configurable: true,
        value: "здравей",
    },

    bar: {      // bar е accessor свойство
        configurable: false,
        get() { // <<------  има getter
            return 10;
        },
        set(value) { // ...  и setter
            console.log("Задаване на `o.bar` на", value);
        },
    },
});

console.log(o.foo);  // "здравей"
console.log(o.bar);  // 10
o.bar = 42;          // "Задаване на `o.bar` на 42"

### Преглед и промяна на поведението на собствените свойства на обекта

JavaScript предоставя вградени методи за преглед и промяна на поведението на свойствата (атрибутите) на обектите.

In [ ]:
// Object.getOwnPropertyDescriptors() - виж ВСИЧКИ property descriptors
let objX = { x: 10, y: 20, z: 33, value: "да" };
objX.arr = [1, 23];

console.log("Всички дескриптори:");
console.log(Object.getOwnPropertyDescriptors(objX));
// {
//   x: { value: 10, writable: true, enumerable: true, configurable: true },
//   y: { value: 20, writable: true, enumerable: true, configurable: true },
//   z: { value: 33, writable: true, enumerable: true, configurable: true },
//   value: { value: "да", writable: true, enumerable: true, configurable: true },
//   arr: { value: [ 1, 23 ], writable: true, enumerable: true, configurable: true }
// }

In [ ]:
// Object.getOwnPropertyDescriptor() - взима мет-описанието за ЕДНО свойство
console.log("Дескриптор за 'z':");
console.log(Object.getOwnPropertyDescriptor(objX, "z"));
// { value: 33, writable: true, enumerable: true, configurable: true }

In [ ]:
// Object.getOwnPropertyNames() - връща ВСИЧКИ собствени свойства (включително неизброими)
console.log("Всички имена на свойства:", Object.getOwnPropertyNames(objX));
// [ "x", "y", "z", "value", "arr" ]

In [ ]:
// Object.defineProperty() - модифицирай property descriptor
// Правене на свойство 'z' неизброимо (скрито от for...in цикли)
Object.defineProperty(objX, "z", { enumerable: false, value: objX.z });

console.log("След като направихме 'z' неизброимо:");
console.log(objX);  // z е скрито от нормалното изброяване
// { x: 10, y: 20, value: "да", arr: [ 1, 23 ] }

console.log("Но 'z' все още съществува:");
console.log(objX.z);  // 33

In [ ]:
// Правене на свойство само за четене (незаписваемо)
Object.defineProperty(objX, "x", { writable: false, value: objX.x });

console.log("След като направихме 'x' само за четене:");
try {
  objX.x = 499;  // хвърля грешка в строг режим
} catch (e) {
  console.log("Грешка:", e.message);
  // Грешка: Cannot assign to read only property 'x' of object
}

### Конструкторна функция, която работи със или без 'new'

In [ ]:
function Cons() { // Тази функция поддържа конструиране както 
    let self;     // с нов контекст, така и без, макар и не препоръчително

    if (this === global || this === undefined) {
        self = Object.create(Cons.prototype);
    } else {
        self = this;
    }

    self.baba = 10;
    return self;
}

let obj1 = new Cons();
let obj2 = Cons();

console.log(obj1.baba);  // 10
console.log(obj2.baba);  // 10
console.log(obj1 instanceof Cons);  // true
console.log(obj2 instanceof Cons);  // true

### Модел на наследяване използвайки Object.create()

In [ ]:
function BaseClass() {
    return this;
}

BaseClass.prototype.val = 10;

function InheriClass() {
    return this;
}

// Настройване на наследяване използвайки Object.create()
// Това позволява да се следва прототипната верига
// където това основно казва нещо като
// InheriClass.prototype = { __proto__ : BaseClass.prototype }
InheriClass.prototype = Object.create(BaseClass.prototype);

let v = new InheriClass();
console.log(v.val);  // 10 (наследено от BaseClass)

## 2. Прототипната верига (обяснено просто)

Представете си, че търсите играчка:
1. Първо, проверявате в собствената си кутия за играчки
2. Ако я няма там, питате по-големия си брат/сестра
3. Ако те нямат, те питат родителите ви
4. Ако родителите нямат, те питат баба и дядо
5. В крайна сметка стигате до края (null)

Точно така работи прототипната верига на JavaScript!

In [ ]:
let grandparent = { surname: "Смит" };
let myparent = Object.create(grandparent);
myparent.job = "Инженер";
let child = Object.create(myparent);
child.name = "Алиса";

console.log(child.name);     // "Алиса" (собствено свойство)
console.log(child.job);      // "Инженер" (от myparent)
console.log(child.surname);  // "Смит" (от grandparent)

// Веригата: child -> parent -> grandparent -> Object.prototype -> null

## 3. Конструкторни функции и прототипи

In [ ]:
function Person(name, age) {
    this.name = name;
    this.age = age;
}

// Добавяне на методи към прототипа (споделени от всички инстанции)
Person.prototype.greet = function() {
    return `Здравей, аз съм ${this.name}`;
};

const alice = new Person("Лиса", 25);
const bob = new Person("Зуг", 30);

console.log(alice.greet());     // "Здравей, аз съм Лиса"
console.log(bob.greet());       // "Здравей, аз съм Зуг"

// И двамата споделят един и същи метод greet
console.log(alice.greet === bob.greet);  // true

## 4. Разбиране на `__proto__` срещу `prototype`

- `prototype`: Свойство на конструкторните функции, дефинира какво ще наследят инстанциите
- `__proto__`: Свойство на обектите, сочи към действителния прототипен обект

Мислете за това по този начин:
- `prototype` е проектът
- `__proto__` е действителната връзка

In [ ]:
function Dog(name) {
    this.name = name;
}

Dog.prototype.bark = function() {
    return "Бау!";
};

const rex = new Dog("Рекс");

console.log(Dog.prototype);     // Dog.prototype е проектът

// rex.__proto__ основно сочи към Dog.prototype
// но може да бъде достъпен директно само в NodeJS 
// докато в Deno има ограничение за докосването му

console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true

// rex може да използва bark заради прототипната верига
console.log(rex.bark());  // "Бау!"

## 5. Object.create() и манипулиране на прототипи

In [ ]:
// Object.create() създава обект с указан прототип
const animal = {
    eat:   function() {    return "Ям...";    }
};

const dog = Object.create(animal);
dog.bark = function() {    return "Бау!";        };

console.log(dog.bark());  // "Бау!" (собствен метод)
console.log(dog.eat());   // "Ям..." (от прототипа)

console.log(Object.getPrototypeOf(dog) === animal);  // true

## 6. Модели на наследяване

In [ ]:
// Родителски конструктор
function Animal(name) {
    this.name = name;
}

Animal.prototype.eat = function() {
    return this.name + ' яде';
};

function Dog(name, breed) {   // Детски конструктор
    Animal.call(this, name);  // Извикване на родителския конструктор
    this.breed = breed;       // но използвайки контекста на Dog (this)
}

if ( false ) {   
    // някои източници правят наследяването така
    // но никога не поправят конструктора, който ще
    // сочи към функцията Animal, ако го оставим така
    Dog.prototype = Object.create(Animal.prototype);

    // затова също трябва да поправим референцията на конструктора, тъй като
    // новосъздаденият прототипен обект няма свойство constructor 
    // и ще се разреши към Animal.constructor свойството
    Dog.prototype.constructor = Dog; 
} else {
    // алтернативно можем просто да кажем, което също е по-очевидно
    Object.setPrototypeOf(Dog.prototype, Animal.prototype);
}

Dog.prototype.bark = function() {
    return this.name + ' казва Бау!';
};

const rex = new Dog("Рекс", "Немска овчарка");
console.log(rex.eat());   // "Рекс яде" (наследено)
console.log(rex.bark());  // "Рекс казва Бау!" (собствен метод)
console.log(Object.getPrototypeOf(rex).constructor)

### Визуален пример на прототипна верига

```
r+--------------------+
|   rex (инстанция)  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|   Dog.prototype    |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Animal.prototype  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Object.prototype  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|        null        |
+--------------------+
```

In [ ]:
// Проверка на веригата
console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true
console.log(Object.getPrototypeOf(Dog.prototype) === Animal.prototype);  // true
console.log(Object.getPrototypeOf(Animal.prototype) === Object.prototype);  // true
console.log(Object.getPrototypeOf(Object.prototype) === null);  // true

### ES6 класове (синтактична захар)

ES6 класовете са просто синтактична захар върху прототипите - те работят по същия начин под капака!

In [ ]:
class ModernAnimal {
    constructor(name) {
        this.name = name;
    }
    
    eat() {
        return `${this.name} яде леко`;
    }
}

class FancyDog extends ModernAnimal {
    constructor(name, breed) {
        super(name);          // Извикване на родителския конструктор
        this.breed = breed;
    }
    
    bark() {
        return this.name + ' казва Зап Зап!';
    }
}

const rex = new FancyDog( "Рекс", "Немски киберонавт");
console.log(rex.eat());       // "Рекс яде"
console.log(rex.bark());      // "Рекс казва Бау!"

// Все още използва прототипи под капака
console.log(Object.getPrototypeOf(rex) === FancyDog.prototype);  // true
console.log(rex.constructor, Object.getPrototypeOf(rex).constructor);

### Mixins

Mixins са начин за добавяне на функционалност към обекти или класове без използване на наследяване. Mixin е обект, който съдържа методи, които да бъдат използвани от други обекти. Можете да използвате `Object.assign()`, за да копирате методите от mixin към прототипа на конструкторна функция или клас.

In [ ]:
const flyMixin = {
  fly() {
    console.log(`${this.name} лети!`);
  }
};

function Bird(name) {
  this.name = name;
}

Object.assign(Bird.prototype, flyMixin);

const eagle = new Bird('Орел');
eagle.fly(); // Орел лети!

## 7. Обобщение

- **Прототипите** позволяват наследяване на обекти в JavaScript
- **Прототипната верига** е начинът, по който JavaScript търси свойства
- **Конструкторните функции** използват `prototype`, за да споделят методи
- **`__proto__`** е действителната връзка, **`prototype`** е проектът
- **ES6 класовете** са синтактична захар върху прототипите
- Разбирането на прототипите е ключово за овладяването на JavaScript!

## 8. Капани и добри практики

Докато прототипите са мощни, има някои чести капани и добри практики, които трябва да имате предвид.

### a) Никога не модифицирайте `Object.prototype`

Модифицирането на `Object.prototype` се счита за лоша практика, защото засяга всички обекти във вашето приложение. Това може да доведе до неочаквано поведение и конфликти с други библиотеки. Това често се нарича 'замърсяване на прототипа'.

In [ ]:
// Лоша практика!
Object.prototype.foo = 'bar';

const myObj = {};
console.log(myObj.foo); // 'bar'

// Това може да счупи for...in цикли
for (const key in myObj) {
    console.log(key); // 'foo'
}

### b) Възстановяване на свойството `constructor`

Когато ръчно задавате прототипа на конструкторна функция, презаписвате нейното свойство `constructor`. Важно е да го възстановите, за да сочи обратно към правилната конструкторна функция. Това гарантира, че `instanceof` и други механизми, които разчитат на свойството constructor, работят правилно. Или използвайте `Object.setPrototypeOf` вместо това.

In [ ]:
function Animal() {}
function Dog() {}

Dog.prototype = Object.create(Animal.prototype);
// Без следващия ред, rex.constructor би бил Animal
Dog.prototype.constructor = Dog;

// или използвайте 
Object.setPrototypeOf(Dog.prototype, Animal.prototype);

const rex = new Dog();
console.log(rex.constructor === Dog); // true

## Упражнения

### Упражнение 1: Поправете счупената прототипна верига

Следният код има счупена прототипна верига. Поправете го така, че `cat.purr()` и `cat.eat()` да работят и двете.

In [ ]:
function Animal() { }
Animal.prototype.munch = function() { console.log('дъвча...') };

function Cat() { }
// Поправете реда по-долу
Cat.prototype = new Animal();

Cat.prototype.purr = function() { console.log('мърка') };

const cat = new Cat();
cat.purr();
cat.munch();

### Упражнение 2: Имплементирайте `SuperArray`

Създайте конструктор `SuperArray`, който наследява от `Array`. Добавете метод към прототипа на `SuperArray`, наречен `last()`, който връща последния елемент на масива.

In [ ]:
function SuperArray(...args) {
  const arr = new Array(...args);
  Object.setPrototypeOf(arr, SuperArray.prototype);
  return arr;
}

SuperArray.prototype = Object.create(Array.prototype);

// Добавете вашия метод тук

const myArr = new SuperArray(1, 2, 3, 4, 5);
console.log(myArr.last()); // 5